https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.1002328

In [62]:
import pandas as pd
import glob
import os
import numpy as np
from zipfile import ZipFile
import tarfile
# import fslnets as fsln
import os
from string import Template
import pickle
import statistics
import community
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('/Users/gracer/Google Drive/HCP_graph/1200/datasets/RESTRICTED_gshearrer_4_19_2018_11_33_34.csv', sep=',')

In [3]:
df.head()

,Subject,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,TestRetestInterval,Race,...,SSAGA_Times_Used_Illicits,SSAGA_Times_Used_Cocaine,SSAGA_Times_Used_Hallucinogens,SSAGA_Times_Used_Opiates,SSAGA_Times_Used_Sedatives,SSAGA_Times_Used_Stimulants,SSAGA_Mj_Use,SSAGA_Mj_Ab_Dep,SSAGA_Mj_Age_1st_Use,SSAGA_Mj_Times_Used
0,100004,24,True,NotTwin,,52259_82122,52259,82122,NaN,White,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1,100206,27,True,NotTwin,,56037_85858,56037,85858,NaN,White,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2,100307,27,True,NotMZ,MZ,51488_81352,51488,81352,NaN,White,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0
3,100408,33,True,MZ,MZ,51730_81594,51730,81594,NaN,White,...,4.0,5.0,5.0,5.0,0.0,0.0,1.0,1.0,2.0,4.0
4,100610,27,True,NotMZ,DZ,52813_82634,52813,82634,NaN,White,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [4]:
list(df.columns)

['Subject',
 'Age_in_Yrs',
 'HasGT',
 'ZygositySR',
 'ZygosityGT',
 'Family_ID',
 'Mother_ID',
 'Father_ID',
 'TestRetestInterval',
 'Race',
 'Ethnicity',
 'Handedness',
 'SSAGA_Employ',
 'SSAGA_Income',
 'SSAGA_Educ',
 'SSAGA_InSchool',
 'SSAGA_Rlshp',
 'SSAGA_MOBorn',
 'Height',
 'Weight',
 'BMI',
 'SSAGA_BMICat',
 'SSAGA_BMICatHeaviest',
 'Blood_Drawn',
 'Hematocrit_1',
 'Hematocrit_2',
 'BPSystolic',
 'BPDiastolic',
 'ThyroidHormone',
 'HbA1C',
 'Hypothyroidism',
 'Hypothyroidism_Onset',
 'Hyperthyroidism',
 'Hyperthyroidism_Onset',
 'OtherEndocrn_Prob',
 'OtherEndocrine_ProbOnset',
 'Menstrual_RegCycles',
 'Menstrual_Explain',
 'Menstrual_AgeBegan',
 'Menstrual_CycleLength',
 'Menstrual_DaysSinceLast',
 'Menstrual_AgeIrreg',
 'Menstrual_AgeStop',
 'Menstrual_MonthsSinceStop',
 'Menstrual_UsingBirthControl',
 'Menstrual_BirthControlCode',
 'FamHist_Moth_Scz',
 'FamHist_Fath_Scz',
 'FamHist_Moth_Dep',
 'FamHist_Fath_Dep',
 'FamHist_Moth_BP',
 'FamHist_Fath_BP',
 'FamHist_Moth_Anx',


In [5]:
myvars=['Subject', 'Age_in_Yrs', 'ZygosityGT', 'Race', 'Ethnicity', 'BMI', 'HbA1C', 'Hypothyroidism',
 'Hyperthyroidism',
 'OtherEndocrn_Prob', 'Mother_ID','Father_ID']

In [6]:
df=df[myvars]

In [7]:
df.shape

(1206, 12)

In [8]:
df=df.loc[df['Hypothyroidism'] != 1]

In [9]:
df.shape

(1202, 12)

In [10]:
df=df.loc[df['Hyperthyroidism'] != 1]

In [11]:
df.shape

(1200, 12)

In [12]:
df=df.loc[df['OtherEndocrn_Prob'] != 1]

In [13]:
df.shape

(1180, 12)

In [14]:
df['BMI'].describe()

count    1179.000000
mean       27.024139
std         5.734938
min        16.480000
25%        22.960000
50%        25.820000
75%        29.990000
max        48.250000
Name: BMI, dtype: float64

In [15]:
df=df.loc[df['BMI'] > 18]

In [16]:
df.shape

(1169, 12)

In [17]:
df=df.dropna(subset=['BMI'])

In [18]:
df.shape

(1169, 12)

In [19]:
df['ZygosityGT'].describe()

count     1169
unique       3
top           
freq       701
Name: ZygosityGT, dtype: object

In [20]:
df['ZygosityGT'].value_counts()

      701
MZ    288
DZ    180
Name: ZygosityGT, dtype: int64

In [21]:
df_mz=df.loc[df['ZygosityGT'] == 'MZ']

In [22]:
df_dz=df.loc[df['ZygosityGT'] == 'DZ']

In [23]:
df_nt=df.loc[df['ZygosityGT'] == ' ']

In [24]:
df_nt.shape

(701, 12)

In [25]:
df_nt['Mother_ID'] = pd.Categorical(df_nt.Mother_ID)
df_nt['Mother_ID'].describe()

/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


count       701
unique      404
top       52385
freq          5
Name: Mother_ID, dtype: int64

In [26]:
df_unt=df_nt.drop_duplicates(['Mother_ID','Father_ID'])

In [27]:
df_unt.shape

(415, 12)

In [28]:
df_unt.loc[df_unt['BMI'] <25 , 'ov_ob'] = 'no'
df_unt.loc[(df_unt['BMI'] >= 25) & (df_unt['BMI'] < 30), 'ov_ob'] = 'ov'
df_unt.loc[df_unt['BMI'] >= 30, 'ov_ob'] = 'ob'

/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
df_unt['ov_ob'].value_counts()

no    177
ov    126
ob    112
Name: ov_ob, dtype: int64

In [30]:
df_unt.to_csv('/Users/gracer/Google Drive/HCP_graph/1200/datasets/not_related.csv', sep=',')

In [31]:
list_int = [os.path.join('node_timeseries/3T_HCP1200_MSMAll_d100_ts2/','%s.txt'%x) for x in df_unt['Subject']]

In [32]:
def tar_heel(X, liist, out):
    tar = tarfile.open(X)
    tar.extractall(out, members=[m for m in tar.getmembers() if m.name in liist])
    tar.close()

In [33]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200'

yarp=os.path.join(basepath,'datasets','HCP_PTN1200')
narp= os.path.join(basepath,'datasets','HCP_PTN1200','graph_analysis')# Macintosh HD⁩ ▸ ⁨Users⁩ ▸ ⁨gracer⁩ ▸ ⁨Google Drive⁩ ▸ ⁨HCP_graph⁩ ▸ ⁨1200⁩ ▸ ⁨datasets⁩ ▸ ⁨HCP_PTN1200⁩
# tar_heel(os.path.join(yarp,'NodeTimeseries_3T_HCP1200_MSMAll_ICAd100_ts2.tar.gz'), list_int, narp)

In [34]:
ovob_dict={'no':{},'ov':{},'ob':{}}

In [35]:
df_unt=df_unt.set_index('Subject')
base_dict=df_unt.to_dict('index')

In [36]:
base_dict

{100004: {'Age_in_Yrs': 24,
  'BMI': 25.49,
  'Ethnicity': 'Not Hispanic/Latino',
  'Father_ID': 82122,
  'HbA1C': 5.4,
  'Hyperthyroidism': nan,
  'Hypothyroidism': nan,
  'Mother_ID': 52259,
  'OtherEndocrn_Prob': nan,
  'Race': 'White',
  'ZygosityGT': ' ',
  'ov_ob': 'ov'},
 100206: {'Age_in_Yrs': 27,
  'BMI': 26.64,
  'Ethnicity': 'Not Hispanic/Latino',
  'Father_ID': 85858,
  'HbA1C': nan,
  'Hyperthyroidism': nan,
  'Hypothyroidism': nan,
  'Mother_ID': 56037,
  'OtherEndocrn_Prob': nan,
  'Race': 'White',
  'ZygosityGT': ' ',
  'ov_ob': 'ov'},
 101006: {'Age_in_Yrs': 35,
  'BMI': 30.27,
  'Ethnicity': 'Not Hispanic/Latino',
  'Father_ID': 81149,
  'HbA1C': 6.0,
  'Hyperthyroidism': 0.0,
  'Hypothyroidism': 0.0,
  'Mother_ID': 51283,
  'OtherEndocrn_Prob': 0.0,
  'Race': 'Black or African Am.',
  'ZygosityGT': ' ',
  'ov_ob': 'ob'},
 101107: {'Age_in_Yrs': 22,
  'BMI': 21.13,
  'Ethnicity': 'Not Hispanic/Latino',
  'Father_ID': 81833,
  'HbA1C': nan,
  'Hyperthyroidism': nan,
  

In [37]:
datapath = os.path.join(basepath,
                    'datasets','HCP_PTN1200','graph_analysis','node_timeseries','3T_HCP1200_MSMAll_d100_ts2')
for key, value in base_dict.items():
#     print(key)
#     print(value['ov_ob'])
    ovob_dict[value['ov_ob']][key]= os.path.join(datapath,'%s.txt'%key)

In [38]:
corr_dict={'no':{},'ov':{},'ob':{}, 'miss':{}}

In [39]:
ovob_dict['ov']

{100004: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/100004.txt',
 100206: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/100206.txt',
 102513: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/102513.txt',
 102614: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/102614.txt',
 105216: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/105216.txt',
 106218: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/106218.txt',
 107725: '/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/10772

In [40]:
#from https://wiki.biac.duke.edu/biac:analysis:resting_pipeline
import numpy as np
import numpy.ma
import nibabel
from scipy import signal
import os, sys, subprocess
import string, random
import re
import networkx as nx
from optparse import OptionParser, OptionGroup
import logging
import math
from scipy import ndimage as nd
import pdb
logging.basicConfig(format='%(asctime)s %(message)s ', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
outpath=os.path.join(basepath,'test')

def duke_corr(corrtxt):
    sub=corrtxt.split('/')[-1].split('.')[0]
    logging.info('starting correlation')
    rmat = os.path.join(outpath,'%s_r_matrix.nii.gz'%sub)
    rtxt = os.path.join(outpath,'%s_r_matrix.csv'%sub)
    zmat = os.path.join(outpath,'%s_zr_matrix.nii.gz'%sub)
    ztxt = os.path.join(outpath,'%s_zr_matrix.csv'%sub)
    maskname = os.path.join(outpath,'%s_mask_matrix.nii.gz'%sub)

    print(corrtxt)
    timeseries = np.loadtxt(corrtxt,unpack=True)
    myres = np.corrcoef(timeseries)
    myres = np.nan_to_num(myres)
#     print(myres)
#     pdb.set_trace()
    #convert corcoef
    zrmaps = 0.5*np.log((1+myres)/(1-myres))
    #find the inf vals on diagonal
    infs = (zrmaps == np.inf).nonzero()

    #replace the infs with 0            
    for idx in range(len(infs[0])):
        zrmaps[infs[0][idx]][infs[1][idx]] = 0

    nibabel.save(nibabel.Nifti1Image(myres,None) ,rmat)
    nibabel.save(nibabel.Nifti1Image(zrmaps,None) ,zmat)
    np.savetxt(ztxt,zrmaps,fmt='%f',delimiter=',')
    np.savetxt(rtxt,myres,fmt='%f',delimiter=',')

    #create a mask for higher level, include everything below diagonal
    mask = np.zeros_like(myres)
    maskx,masky = mask.shape

    for idx in range(maskx):
        for idy in range(masky):
            if idx > idy:
                mask[idx][idy] = 1

    nibabel.save(nibabel.Nifti1Image(mask,None) ,maskname)


    #check for the resulting files
    for fname in [rmat, zmat, maskname, ztxt, rtxt]:
        if os.path.isfile( fname ):
            logging.info('correlation matrix finished : ' + fname)
        else:
            logging.info('correlation failed')
            raise SystemExit()     
    return(myres)

In [41]:
for key, value in ovob_dict.items():
    print(key)
    for k, v in value.items():
        print(k)
        if os.path.exists(v):
            corr_dict[key][k]=duke_corr(v)
        else:
            print('no %s'%k)
            corr_dict['miss'][k]=v
        

11/12/2019 04:29:07 PM starting correlation 


no
101107
/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/101107.txt


/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101107_r_matrix.nii.gz 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101107_zr_matrix.nii.gz 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101107_mask_matrix.nii.gz 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101107_zr_matrix.csv 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101107_r_matrix.csv 
11/12/2019 04:29:08 PM starting correlation 


101309
/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/101309.txt


11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101309_r_matrix.nii.gz 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101309_zr_matrix.nii.gz 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101309_mask_matrix.nii.gz 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101309_zr_matrix.csv 
11/12/2019 04:29:08 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/101309_r_matrix.csv 
11/12/2019 04:29:09 PM starting correlation 


...

11/12/2019 04:33:21 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/894774_r_matrix.nii.gz 
11/12/2019 04:33:21 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/894774_zr_matrix.nii.gz 
11/12/2019 04:33:21 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/894774_mask_matrix.nii.gz 
11/12/2019 04:33:21 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/894774_zr_matrix.csv 
11/12/2019 04:33:21 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/894774_r_matrix.csv 
11/12/2019 04:33:21 PM starting correlation 


915554
no 915554
937160
/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/937160.txt


11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/937160_r_matrix.nii.gz 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/937160_zr_matrix.nii.gz 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/937160_mask_matrix.nii.gz 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/937160_zr_matrix.csv 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/937160_r_matrix.csv 
11/12/2019 04:33:22 PM starting correlation 


940553
no 940553
947668
/Users/gracer/Google Drive/HCP_graph/1200/datasets/HCP_PTN1200/graph_analysis/node_timeseries/3T_HCP1200_MSMAll_d100_ts2/947668.txt


11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/947668_r_matrix.nii.gz 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/947668_zr_matrix.nii.gz 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/947668_mask_matrix.nii.gz 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/947668_zr_matrix.csv 
11/12/2019 04:33:22 PM correlation matrix finished : /Users/gracer/Google Drive/HCP_graph/1200/test/947668_r_matrix.csv 


In [42]:
len(list(corr_dict['ob'].keys()))

78

In [43]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    graph_dict={}
    partition_dict={}
    for key, values in list_o_data.items():
            ########################################
            edge_dict.setdefault(key, []).append(values)
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            #partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            #partition_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'graphs':graph_dict}) #'modules':partition_dict})

In [44]:
graphs={'no':{},'ov':{},'ob':{}}
for key, value in corr_dict.items():
    print(key)
    if key == 'miss':
        print('skip')
    else:
        graphs[key]=make_graphs(value, 'positive',0)

no
ov
ob
miss
skip


In [45]:
for key, value in graphs.items():  
    o=[]
    for k,v in value.items():
        if k == 'mean_FC':
            for i, j in v.items():
                o.append(j[0])
    print('this is %s mean %f'%(key,statistics.mean(o)))
    print('this is %s sd %f'%(key,statistics.stdev(o)))

this is no mean 0.128643
this is no sd 0.022119
this is ov mean 0.124296
this is ov sd 0.020359
this is ob mean 0.122282
this is ob sd 0.019700


If this value is low (<0.1) then you don't need to threshold the graph? 

In [46]:
#make the function to combine
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.median(x, axis=(2))
    return(mu)

In [47]:
total_graphs={'no':{}, 'ov':{},'ob':{}}
for key, value in graphs.items():
    print(key)
    print(value.keys())
    total_graphs[key]=make_total_graphs(value['correlations'])
    

no
dict_keys(['edges', 'correlations', 'mean_FC', 'graphs'])
ov
dict_keys(['edges', 'correlations', 'mean_FC', 'graphs'])
ob
dict_keys(['edges', 'correlations', 'mean_FC', 'graphs'])


In [48]:
total_graphs['no']

array([[1.        , 0.08913704, 0.37471215, ..., 0.02521382, 0.03279483,
        0.04459487],
       [0.08913704, 1.        , 0.10879102, ..., 0.03347942, 0.03568274,
        0.03360623],
       [0.37471215, 0.10879102, 1.        , ..., 0.03913156, 0.05719495,
        0.0504471 ],
       ...,
       [0.02521382, 0.03347942, 0.03913156, ..., 1.        , 0.04700598,
        0.05547095],
       [0.03279483, 0.03568274, 0.05719495, ..., 0.04700598, 1.        ,
        0.06277449],
       [0.04459487, 0.03360623, 0.0504471 , ..., 0.05547095, 0.06277449,
        1.        ]])

In [50]:
def ges(mu):
    G=nx.from_numpy_matrix(np.matrix(mu))
    partition = community.best_partition(G)
    vals = list(partition.values())
    nx.set_node_attributes(G, partition, 'modules')
    return((partition,vals, G))

In [51]:
from sklearn.metrics.pairwise import euclidean_distances
# distance between rows of X
no_test=euclidean_distances(total_graphs['no'], total_graphs['no'])
ov_test=euclidean_distances(total_graphs['ov'], total_graphs['ov'])
ob_test=euclidean_distances(total_graphs['ob'], total_graphs['ob'])

In [52]:
euNO = nx.from_numpy_matrix(np.matrix(no_test))

In [53]:
no_test.mean()

1.6325626661926587

In [54]:
ov_test.mean()

1.626911695968336

In [55]:
ob_test.mean()

1.6200658099887362

In [56]:
graph_parts={'no':{},'ov':{}, 'ob':{}}
for key, value in total_graphs.items():
    [partion,vals, graph]=ges(value)
    graph_parts[key]=[partion,vals, graph]

In [59]:
graph_parts['no'][2].node(data=True)

NodeDataView({0: {'modules': 0}, 1: {'modules': 1}, 2: {'modules': 0}, 3: {'modules': 0}, 4: {'modules': 2}, 5: {'modules': 2}, 6: {'modules': 2}, 7: {'modules': 0}, 8: {'modules': 2}, 9: {'modules': 2}, 10: {'modules': 0}, 11: {'modules': 1}, 12: {'modules': 0}, 13: {'modules': 3}, 14: {'modules': 4}, 15: {'modules': 0}, 16: {'modules': 0}, 17: {'modules': 2}, 18: {'modules': 0}, 19: {'modules': 0}, 20: {'modules': 4}, 21: {'modules': 2}, 22: {'modules': 5}, 23: {'modules': 3}, 24: {'modules': 0}, 25: {'modules': 2}, 26: {'modules': 1}, 27: {'modules': 0}, 28: {'modules': 2}, 29: {'modules': 0}, 30: {'modules': 2}, 31: {'modules': 0}, 32: {'modules': 3}, 33: {'modules': 5}, 34: {'modules': 1}, 35: {'modules': 2}, 36: {'modules': 2}, 37: {'modules': 1}, 38: {'modules': 0}, 39: {'modules': 3}, 40: {'modules': 4}, 41: {'modules': 2}, 42: {'modules': 4}, 43: {'modules': 2}, 44: {'modules': 4}, 45: {'modules': 1}, 46: {'modules': 4}, 47: {'modules': 1}, 48: {'modules': 4}, 49: {'modules': 

In [66]:
def jenny_graph(graph):
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    nodes, color = zip(*nx.get_node_attributes(graph,'modules').items()) #if your modules are named different change here
    nodes, positions = zip(*nx.get_node_attributes(graph,'modules').items())
    #positions
    positions=nx.circular_layout(graph) #this is defining a circluar graph, if you want a different one you change the circular part of this line
    
    #Figure size
    plt.figure(figsize=(40,25))
    
    
    #draws nodes
    color = np.array(color)
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    y=nx.draw_networkx_nodes(graph,positions, 
                           node_color=color,
                           node_size=40,
                           alpha=0.8, 
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(graph, positions, font_size=10, 
                            font_family='sans-serif', fontweight = 'bold')
    
    
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.bwr #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    wt=weights*5
    M=wt.max()
    m=wt.min()
    x=nx.draw_networkx_edges(graph, positions, edge_list=edges, style='solid', width = wt, edge_color = wt,
                           cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)
    
    #format the colorbar
    node_bar=plt.colorbar(y)
    edge_bar=plt.colorbar(x)

    node_bar.set_label('Modularity',fontsize = 25)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)

    plt.axis('off')
    plt.title("Modularity and Edge Weights of Average Graph", fontsize = 30)
    #plt.savefig(os.path.join(basepath,"betaseries_bevel/5_analysis/modularity_circle_reward.png", format="PNG")
    plt.show()

In [68]:
def threshold(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

In [70]:
graph_parts['no'][2].edges(data=True)

EdgeDataView([(0, 0, {'weight': 1.0}), (0, 1, {'weight': 0.08913704421775988}), (0, 2, {'weight': 0.37471215222372206}), (0, 3, {'weight': 0.33457858709782407}), (0, 4, {'weight': 0.08074017039463634}), (0, 5, {'weight': 0.0880022145469172}), (0, 6, {'weight': 0.06411464309289774}), (0, 7, {'weight': 0.1023867662495238}), (0, 8, {'weight': 0.12320954864410769}), (0, 9, {'weight': 0.07519453000694787}), (0, 10, {'weight': 0.303406593377481}), (0, 11, {'weight': 0.07608906935304391}), (0, 12, {'weight': 0.1504742209688523}), (0, 13, {'weight': 0.07599178497007149}), (0, 14, {'weight': 0.1634599723235906}), (0, 15, {'weight': 0.18058769787824142}), (0, 16, {'weight': 0.13614212290013378}), (0, 17, {'weight': 0.09491041732770476}), (0, 18, {'weight': 0.18411140285052613}), (0, 19, {'weight': 0.13929232453231993}), (0, 20, {'weight': 0.09861597218665562}), (0, 21, {'weight': 0.05662827010454825}), (0, 22, {'weight': 0.11702168241745936}), (0, 23, {'weight': 0.08129613237435904}), (0, 24, {'

In [77]:
no=threshold(graph_parts['no'][2], 'positive',.5)

In [79]:
ov=threshold(graph_parts['ov'][2], 'positive',.5)

In [81]:
ob=threshold(graph_parts['ob'][2], 'positive',.5)

In [82]:
jenny_graph(ob)

/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:585: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  and cb.iterable(edge_color) \
/Users/gracer/anaconda3/envs/py36/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:595: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  for c in edge_color]):


In [105]:
mod_ob = community.induced_graph(graph_parts['ob'][0], graph_parts['ob'][2])

In [100]:
mod_ov = community.induced_graph(graph_parts['ov'][0], graph_parts['ov'][2])

In [102]:
mod_no = community.induced_graph(graph_parts['no'][0], graph_parts['no'][2])

In [106]:
def module_fig(G, Type):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())
    
    
    positions=nx.circular_layout(G)
    plt.figure(figsize=(25,20))
    
    color = np.array(list(G.nodes))
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    nx.draw_networkx_nodes(G,positions, 
                           node_color=color, 
                           #node_size=size,
                           alpha=1.0, 
                           cmap= 'Spectral',
                           vmin=cm,vmax=cM )
   
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, font_family='sans-serif')
    wt=np.array(weights)/5
    x=nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = wt, edge_color = weights)
   
    edge_bar=plt.colorbar(x)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)
    
    plt.title("Module Connectivity Weights %s"%Type, fontsize = 30)
    #plt.savefig(os.path.join(basepath,"betaseries_bevel/5_analysis/results/modularity_edges_reward_weighted.png"), format="PNG")
    plt.axis('off')
    plt.show()

In [108]:
def make_node_stats(list_o_data, direction, min_cor):
    clustering_dict ={}
    centrality_dict ={}
       
    for key, values in list_o_data.items():
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            tG = create_corr_network_5(G, direction,min_cor)
            ########################################
            clustering = nx.clustering(tG, weight=True)
            ########################################
            clustering_dict.setdefault(key, []).append(clustering)
            ########################################
            centrality = nx.betweenness_centrality(tG, weight=True)
            ########################################
            centrality_dict.setdefault(key, []).append(centrality)
            ########################################
    return({'clustering_coeff':clustering_dict, 'btn_centrality':centrality_dict})

In [ ]:
# GRAPHS={'corr':corr_dict,'total':total_graphs, 'graphs':graphs, 'parts':graph_parts}

In [ ]:
# pickle.dump(GRAPHS, open(os.path.join(basepath, 'test/GraphsGES'), 'wb'), protocol=4)

In [ ]:
# basepath='/Users/gracer/Google Drive/HCP_graph/1200'

# with open(os.path.join(basepath, 'test/GraphsGES'), 'rb') as pickle_file:
#     try:
#         while True:
#             GRAPHS = pickle.load(pickle_file)
# #             print (GRAPHS)
#     except EOFError:
#         pass